In [ ]:
using CUDA
using Flux              # the julia ml library


In [ ]:
# fake data
N = 60000

X_true = randn((28, 28, N))
y_true = rand(1:10, N)

function preprocess(X, y)
    Xs = Float32.(X) |> Flux.flatten
    ys = Flux.onehotbatch( Float32.(y), 1:10 )
    
    return (Xs, ys)
end

h, v, N = size(X_true); d = h * v
X, y = preprocess(X_true, y_true);

In [ ]:
d, N, size(X), size(y)

In [ ]:
#CUDA.allowscalar(false)

X_d = X |> gpu   # data on gpu
y_d = y |> gpu   # data on gpu

# model configuration
nInputs  = d
nOutputs = 10
model = Flux.Dense(nInputs, nOutputs, relu) |> gpu   # weights on gpu

losses(X, y)    = Flux.crossentropy( model(X), y )
modelParameters = Flux.params(model)
dataLoader = @CUDA.allowscalar Flux.DataLoader((X_d, y_d), batchsize=10000)


In [ ]:
# Flux.train!(loss, params, data, opt; cb)
@time Flux.train!(losses, modelParameters, dataLoader, Descent())